In [37]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

In [41]:
def create_linear_model(sheet_name):
    kidney_df_cleaned = pd.read_csv("merged/"+sheet_name)
    Y = kidney_df_cleaned["Patient Survival Rate"]
    X = kidney_df_cleaned.drop(columns=['Unnamed: 0', 'N', 'Patient Survival Rate'])
    X = pd.get_dummies(X)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=1)
    
    lr_model = LinearRegression()
    # Fitting the model
    lr_model.fit(X_train, Y_train)
    # Making predictions using the testing data
    predictions = lr_model.predict(X_test)
    
    score = lr_model.score(X_train, Y_train, sample_weight=None)
    r2 = r2_score(Y_test, predictions)
    print(f"Accuracy Score for {sheet_name}: {score}")
    print(f"r2 Score for {sheet_name}: {r2}")

In [31]:
# Read the CSV file into a Pandas DataFrame
kidney_df_cleaned = pd.read_csv("merged/15_Year_cleaned.csv")
kidney_df_cleaned

,Unnamed: 0,Transplant Year,Age Group,N,Graft Survival Rate,Patient Survival Rate,All ABO,O,A,B,...,Retransplant/Graft Failure,Tubular and Interstitial Diseases,"White, Non-Hispanic","Black, Non-Hispanic",Hispanic/Latino,Unknown,"Asian, Non-Hispanic","American Indian/Alaska Native, Non-Hispanic","Pacific Islander, Non-Hispanic","Multiracial, Non-Hispanic"
0,0,2007,<1 Year,1,0.00,0.00,2,1,1,0,...,0.0,0.0,180,55,77,0,20,2,4,5
1,1,2007,1-5 Years,154,50.23,90.34,162,88,49,20,...,0.0,13.0,291,95,118,0,29,6,2,6
2,2,2007,6-10 Years,131,44.26,84.95,141,75,36,23,...,6.0,12.0,146,50,63,0,12,0,1,1
3,3,2007,11-17 Years,425,29.14,80.48,491,248,172,52,...,31.0,40.0,406,161,186,0,31,7,1,1
4,4,2007,18-34,1847,42.51,76.32,2265,1074,809,292,...,210.0,110.0,1778,803,588,0,133,23,14,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,155,1988,11-17 Years,271,27.59,80.51,407,182,163,44,...,3.0,62.0,356,103,62,3,11,4,0,0
156,156,1988,18-34,2076,33.78,62.24,2794,1286,1060,322,...,10.0,235.0,2419,603,303,3,83,16,7,0
157,157,1988,35-49,2579,27.71,42.55,3289,1486,1274,383,...,8.0,201.0,3196,838,291,5,109,43,1,0
158,158,1988,50-64,1654,19.17,21.72,1926,876,730,232,...,2.0,112.0,2487,490,179,6,60,20,3,0


In [32]:
#kidney_df_cleaned["Patient_Survival_status"] = good_survival
Y = kidney_df_cleaned["Patient Survival Rate "]
X = kidney_df_cleaned.drop(columns=['Unnamed: 0', 'N', 'Patient Survival Rate '])

In [33]:
X = pd.get_dummies(X)
X

,Transplant Year,Graft Survival Rate,All ABO,O,A,B,AB,"Congenital, Rare, Familial, and Metaboli",Diabetes,Glomerular Disease,...,"Multiracial, Non-Hispanic",Age Group_1-5 Years,Age Group_11-17 Years,Age Group_18-34,Age Group_35-49,Age Group_50-64,Age Group_6-10 Years,Age Group_65+,Age Group_<1 Year,Age Group_Not Reported
0,2007,0.00,2,1,1,0,0,1.0,0.0,0.0,...,5,0,0,0,0,0,0,0,1,0
1,2007,50.23,162,88,49,20,5,54.0,0.0,11.0,...,6,1,0,0,0,0,0,0,0,0
2,2007,44.26,141,75,36,23,7,34.0,0.0,32.0,...,1,0,0,0,0,0,1,0,0,0
3,2007,29.14,491,248,172,52,19,89.0,0.0,168.0,...,1,0,1,0,0,0,0,0,0,0
4,2007,42.51,2265,1074,809,292,90,107.0,137.0,892.0,...,12,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,1988,27.59,407,182,163,44,18,76.0,0.0,177.0,...,0,0,1,0,0,0,0,0,0,0
156,1988,33.78,2794,1286,1060,322,125,88.0,593.0,1220.0,...,0,0,0,1,0,0,0,0,0,0
157,1988,27.71,3289,1486,1274,383,146,32.0,780.0,1104.0,...,0,0,0,0,1,0,0,0,0,0
158,1988,19.17,1926,876,730,232,88,2.0,333.0,561.0,...,0,0,0,0,0,1,0,0,0,0


In [34]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=1)
X_train.shape

(120, 35)

In [35]:
# Create a random forest classifier
lr_model = LinearRegression()
# Fitting the model
lr_model.fit(X_train, Y_train)
# Making predictions using the testing data
predictions = lr_model.predict(X_test)
predictions

array([61.66379163, 88.00382004, 33.31971425, 78.5941878 , 89.56009599,
       80.06109651, 76.65181999, 82.21486892, 42.93016183, 83.60339768,
       86.73703884, 32.2653305 , 89.86599731, 86.98133567, 80.99983659,
       85.39027145, 86.1030856 , 31.73792674, 66.74729346, 70.33973115,
       82.98164388, 74.37427085, 83.40210966, 58.87115266, 72.24980361,
       13.53250689, -0.41934001, 88.82052332,  9.99578692, 58.34736929,
        9.87891239, 11.40189324, 10.57936393, 79.74100918, 83.82346001,
       30.19567892, 14.59144899, 56.95410945, 87.03782091,  0.40333416])

In [36]:
score = lr_model.score(X_train, Y_train, sample_weight=None)
r2 = r2_score(Y_test, predictions)
print(f"Accuracy Score : {score}")
print(f"r2 Score : {r2}")

Accuracy Score : 0.996528616564367
r2 Score : 0.9470567506571431


In [43]:
for sheet in ['1_Year_cleaned.csv', '3_Year_cleaned.csv', '5_Year_cleaned.csv', '10_Year_cleaned.csv', '15_Year_cleaned.csv', '20_Year_cleaned.csv']:
    create_linear_model(sheet)

Accuracy Score for 1_Year_cleaned.csv: 0.9995454531926082
r2 Score for 1_Year_cleaned.csv: 0.9985563213311694
Accuracy Score for 3_Year_cleaned.csv: 0.9991055295440959
r2 Score for 3_Year_cleaned.csv: 0.9977813635074265
Accuracy Score for 5_Year_cleaned.csv: 0.998681960585655
r2 Score for 5_Year_cleaned.csv: 0.9965273821084251
Accuracy Score for 10_Year_cleaned.csv: 0.9982129258191748
r2 Score for 10_Year_cleaned.csv: 0.9962123188633075
Accuracy Score for 15_Year_cleaned.csv: 0.996528616564367
r2 Score for 15_Year_cleaned.csv: 0.9470567506571431
Accuracy Score for 20_Year_cleaned.csv: 0.9952663840997822
r2 Score for 20_Year_cleaned.csv: 0.9898742405681381
